In [2]:
# from google.colab import drive
# drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# %cd /content/drive/MyDrive/Proactiveware

/content/drive/MyDrive/Proactiveware


In [4]:
# !pwd

/content/drive/MyDrive/Proactiveware


In [1]:
# !pip install geopandas
# !pip install earthengine-api
# !pip install folium
# # !pip install sentinelhub
# !pip install gpd
# !pip install georaster
# !pip install folium Pillow
# !pip install geemap


In [ ]:
# !pip create -n gee python=3.8
# !pip activate gee
# !pip  install geopandas
# !pip install mamba 
# mamba install geemap -c conda-forge

In [ ]:
# import io.StringIO

import ee
# Trigger the authentication flow.
ee.Authenticate()
# import geopandas as gpd
import matplotlib.pyplot as plt
import folium
# from folium import plugins

# import sentinelhub
# import gpd

# from geopandas import GeoSeries, GeoDataFrame
import json
import time 

# import georaster

# import geemap



AttributeError: module 'ee' has no attribute 'Authenticate'

In [3]:
# import io.StringIO

import ee
# Trigger the authentication flow.
ee.Authenticate()
# import geopandas as gpd
import matplotlib.pyplot as plt
import folium
# from folium import plugins

# import sentinelhub
# import gpd

# from geopandas import GeoSeries, GeoDataFrame
import json
import time 

# import georaster

# import geemap



AttributeError: module 'ee' has no attribute 'Authenticate'

### Authenticate and initialize

Run the `ee.Authenticate` function to authenticate your access to Earth Engine servers and `ee.Initialize` to initialize it. Upon running the following cell you'll be asked to grant Earth Engine access to your Google account. Follow the instructions printed to the cell.

In [2]:


# Initialize the library.
ee.Initialize()

AttributeError: module 'ee' has no attribute 'Authenticate'

## Test the API

Test the API by printing the elevation of Mount Everest.

In [7]:
# Print the elevation of Mount Everest.
dem = ee.Image('USGS/SRTMGL1_003')
xy = ee.Geometry.Point([86.9250, 27.9881])
elev = dem.sample(xy, 30).first().get('elevation').getInfo()
print('Mount Everest elevation (m):', elev)

Mount Everest elevation (m): 8729


# Layer for folium

In [10]:
# Add custom basemaps to folium
basemaps = {
    'Google Maps': folium.TileLayer(
        tiles = 'https://mt1.google.com/vt/lyrs=m&x={x}&y={y}&z={z}',
        attr = 'Google',
        name = 'Google Maps',
        overlay = True,
        control = True
    ),
    'Google Satellite': folium.TileLayer(
        tiles = 'https://mt1.google.com/vt/lyrs=s&x={x}&y={y}&z={z}',
        attr = 'Google',
        name = 'Google Satellite',
        overlay = True,
        control = True
    ),
    'Google Terrain': folium.TileLayer(
        tiles = 'https://mt1.google.com/vt/lyrs=p&x={x}&y={y}&z={z}',
        attr = 'Google',
        name = 'Google Terrain',
        overlay = True,
        control = True
    ),
    'Google Satellite Hybrid': folium.TileLayer(
        tiles = 'https://mt1.google.com/vt/lyrs=y&x={x}&y={y}&z={z}',
        attr = 'Google',
        name = 'Google Satellite',
        overlay = True,
        control = True
    ),
    'Esri Satellite': folium.TileLayer(
        tiles = 'https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}',
        attr = 'Esri',
        name = 'Esri Satellite',
        overlay = True,
        control = True
    )
}

# plot

In [117]:
geometry

{'coordinates': [[[54.360723589073814, 31.900557381839263],
   [54.360723589073814, 31.89907003779554],
   [54.36249344408964, 31.89907003779554],
   [54.36249344408964, 31.900557381839263],
   [54.360723589073814, 31.900557381839263]]],
 'type': 'Polygon'}

In [11]:

# Define your area of interest as a GeoDataFrame or read it from a GeoJSON file
# gdf = gpd.read_file('geojason/selected_polygon.geojson')


# Opening a GeoJSON file
with open('geojason/selected_polygon.geojson') as f:
    data = json.load(f)

# Accessing the features
features = data['features']

# Convert the geometry column to a list of lists (required format for Earth Engine)
# geometry = gdf.geometry.apply(lambda x: x.__geo_interface__['coordinates'][0]).tolist()
geometry = data['features'][0]['geometry']["geometries"][1]["coordinates"]
# print(geometry)

# Convert the list of lists to an Earth Engine Polygon object
polygon = ee.Geometry.Polygon(geometry)
# print(polygon)


# Define the Sentinel-2 product collection and filter it by date and cloud cover
collection = ee.ImageCollection('COPERNICUS/S2_SR') \
    .filterBounds(polygon) \
    .filterDate('2021-01-01', '2021-03-31') \
    .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 10))

# Select the first image in the filtered collection
image = ee.Image(collection.first())


# display(image.select(['B4', 'B3', 'B2']))# Get the RGB bands to visualize
rgb_bands = ['B4', 'B3', 'B2']

# Convert the image to 8-bit RGB
vis_params = {
    'min': 0,
    'max': 255,
    'bands': rgb_bands
}
image_rgb = image.visualize(**vis_params)

# Define map center and zoom level
centroid = geometry[0][0]
map_center = [centroid[1], centroid[0]]
zoom_level = 100

# Create a folium map object
# my_map = folium.Map(location=map_center, zoom_start=zoom_level)

# Set visualization parameters.
vis_params = {
  'min': 0,
  'max': 4000,
  # 'palette': ['006633', 'E5FFCC', '662A00', 'D8D8D8', 'F5F5F5']
  }

# Create a folium map object.
my_map = folium.Map(location=map_center, zoom_start=100)
# Add the polygon geometry to the map
folium.GeoJson(
    data=polygon.getInfo(),
    name='Polygon',
    style_function=lambda feature: {
        'fillColor': None,
        'color': 'red',
        'weight': 2
    }
).add_to(my_map)
# Add custom basemaps
basemaps['Google Maps'].add_to(my_map)
basemaps['Google Satellite Hybrid'].add_to(my_map)
display(my_map)

# get image from google map

In [ ]:
!pip install turfpy

In [127]:

#  pip install turfpy


from turfpy.measurement import bbox
import math
from PIL import Image
import requests
import os
path = '/content/drive/MyDrive/Proactiveware/figures'

def num2deg(xtile, ytile, zoom):
    n = 2.0 ** zoom
    lon_deg = xtile / n * 360.0 - 180.0
    lat_rad = math.atan(math.sinh(math.pi * (1 - 2 * ytile / n)))
    lat_deg = math.degrees(lat_rad)
    return (lat_deg, lon_deg)

def getXY(lng, lat, zoom):
    tile_size = 256
    numTiles = 1 << zoom
    point_x = (tile_size/ 2 + lng * tile_size / 360.0) * numTiles // tile_size
    sin_y = math.sin(lat * (math.pi / 180.0))
    point_y = ((tile_size / 2) + 0.5 * math.log((1+sin_y)/(1-sin_y)) * -(tile_size / (2 * math.pi))) * numTiles // tile_size
    return int(point_x), int(point_y)


def download_im(geometry, zoom):
    num = 0
    header = {'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/51.0.2704.103 Safari/537.36'}
    minlon, minlat, maxlon, maxlat = bbox(geometry)
    print(minlon, minlat, maxlon, maxlat)

    start_x, start_y = getXY(minlon, maxlat, zoom)
    end_x, end_y = getXY(maxlon, minlat, zoom)


    start_x -=1
    start_y -=1

    end_x +=1
    end_y +=1


    w = (end_x+1 - start_x) * 256
    h = (end_y+1 - start_y) * 256
    result = Image.new("RGB", (w, h))

    for x in range(start_x, end_x+1):
        for y in range(start_y, end_y+1):
            print(f"donwloading {num}/{((end_x+1) - (start_x)) * ((end_y+1) - (start_y)) }")
            
            while True:
                try:
                    url = "http://mt1.google.com/vt/lyrs=y&x={}&y={}&z={}".format(x,y, zoom)
                    raw = requests.get(url, stream=True, headers=header).raw
                    break
                except:
                    continue
            
            i = Image.open(raw)
            x_paste = (x - start_x) * 256
            y_paste = h - (end_y+1 - y) * 256
            result.paste(i, (x_paste, y_paste))
            num +=1
    name = os.path.join(path, "map_{}_{}_{}_{}.png".format(minlat, minlon, maxlat, maxlon))
    result.save(os.path.join (name))
    return result, start_x, start_y, end_x, end_y, w, h


# geometry = {
#         "coordinates": [
#           [
#             [
#               54.360723589073814,
#               31.900557381839263
#             ],
#             [
#               54.360723589073814,
#               31.89907003779554
#             ],
#             [
#               54.36249344408964,
#               31.89907003779554
#             ],
#             [
#               54.36249344408964,
#               31.900557381839263
#             ],
#             [
#               54.360723589073814,
#               31.900557381839263
#             ]
#           ]
#         ],
#         "type": "Polygon"
#       }

geometry = {'coordinates': [[[-94.57260260212433, 39.14019525724463],
       [-94.57071164476883, 39.14019525724463],
       [-94.57071164476883, 39.141118918340055],
       [-94.57260260212433, 39.141118918340055],
       [-94.57260260212433, 39.14019525724463]]],
     'type': 'Polygon'}
# zoom = 18
zoom = 19
download_im(geometry, zoom)

-94.57260260212433 39.14019525724463 -94.57071164476883 39.141118918340055
donwloading 0/24
donwloading 1/24
donwloading 2/24
donwloading 3/24
donwloading 4/24
donwloading 5/24
donwloading 6/24
donwloading 7/24
donwloading 8/24
donwloading 9/24
donwloading 10/24
donwloading 11/24
donwloading 12/24
donwloading 13/24
donwloading 14/24
donwloading 15/24
donwloading 16/24
donwloading 17/24
donwloading 18/24
donwloading 19/24
donwloading 20/24
donwloading 21/24
donwloading 22/24
donwloading 23/24


(<PIL.Image.Image image mode=RGB size=1536x1024 at 0x7F3375AC2B00>,
 124411,
 200106,
 124416,
 200109,
 1536,
 1024)

In [121]:
geometry

[[[-94.57260260212433, 39.14019525724463],
  [-94.57071164476883, 39.14019525724463],
  [-94.57071164476883, 39.141118918340055],
  [-94.57260260212433, 39.141118918340055],
  [-94.57260260212433, 39.14019525724463]]]

In [122]:
data['features']

[{'geometry': {'geodesic': False,
   'geometries': [{'coordinates': [-94.57337904580696, 39.14119634269676],
     'type': 'Point'},
    {'coordinates': [[[-94.57260260212433, 39.14019525724463],
       [-94.57071164476883, 39.14019525724463],
       [-94.57071164476883, 39.141118918340055],
       [-94.57260260212433, 39.141118918340055],
       [-94.57260260212433, 39.14019525724463]]],
     'type': 'Polygon'}],
   'type': 'GeometryCollection'},
  'id': '0',
  'properties': {},
  'type': 'Feature'}]